Install cdsapi package to download our data. We will need xarray to concatenate the data. netCDF4, eccodes, cfgrib, ecmwflibs, and netcdf4 are needed to open the GRIB file and also convert to NetCDF.

When you wish to execute shell commands in a jupyter notebook, start the command with "!". I am using a virtual environment to run this notebook, so it won't install the packages globally.

In [5]:
! python -m pip install "cdsapi>=0.7.4" xarray pandas eccodes cfgrib ecmwflibs netcdf4


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cdsapi
import xarray as xr

In [2]:
c = cdsapi.Client()

2025-05-25 12:09:00,560 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-25 12:09:00,562 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


As of writing this, due to the recent CDS server migrations and slow queues for downloading the data, we can only download a dataset one month at a time. Thus, we will have to loop through every month and year we wish to download. **Important**: Make sure to change the range in 'years_list' to the decades you wish to download. For example, if you're downloading 1951 - 1960, then the range would be range(1951, 1961). In this notebook, I decided to download city data from Ulaanbaatar, Mongolia. The coordinates are 30.2672° N, 97.7431° W, respectively, or 30.3 and -97.7, respectively. **Important**: Since our dataset is specific to land data, it's likely that if the region includes any oceans or large bodies of water, the temperature recorded at those pixels will be inaccurate or missing.

The link to the dataset: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=overview 

Dataset name: ERA5-Land hourly data from 1950 to present

In [3]:
years_list = [str(year) for year in range(2008, 2025)] # 1970, 1980
leap_years = [str(year) for year in range(2008, 2025, 4)]

make_two_digits = ['01', '02', '03', '04', '05', '06', '07', '08', '09'] # API accepts single digit days in this format

thirty_one = make_two_digits + [str(year) for year in range(10, 32)]
thirty = make_two_digits + [str(year) for year in range(10, 31)]
feb_days_leap = make_two_digits + [str(year) for year in range(10, 30)]
feb_days_non_leap = make_two_digits + [str(year) for year in range(10, 29)]

months_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'] # API aceepts months in this format
month_days = {
	'01':thirty_one,
	'02':feb_days_non_leap,
	'03':thirty_one,
	'04':thirty,
	'05':thirty_one,
	'06':thirty,
	'07':thirty_one,
	'08':thirty_one,
	'09':thirty,
	'10':thirty_one,
	'11':thirty,
	'12':thirty_one,
}

# Currently downloading data for a model for Houston, TX.
city_long = -95.4
city_lat = 29.7
city_name = 'Houston'

# Region boundaries for the capital. Will produce a 5x5 grid of values each time step. The spatial resolution of the dataset is 0.1° x 0.1°.
bottom_lat = str(round(city_lat - 0.2, 1))
top_lat = str(round(city_lat + 0.2, 1))
left_long = str(round(city_long - 0.2, 1))
right_long = str(round(city_long + 0.2, 1))

# IMPORTANT: Change 'ds_path' to wherever you originally store the data
ds_path = 'C:/Users/sguti/WW-Data/'

In [4]:
# datasets = []

# for year in years_list:
	
# 	for month in months_list:

# 		# Take care of leap years
# 		if month == '02' and year in leap_years:
# 			month_days['02'] = feb_days_leap
# 		else:
# 			month_days['02'] = feb_days_non_leap
			
# 		grib_filename = ds_path + city_name + '_' + year + '_' + month + '.grib'
		
# 		datasets.append({
# 			'variable': [
# 				'2m_temperature', 
# 				'2m_dewpoint_temperature', 
# 				'surface_pressure', 
# 				'10m_u_component_of_wind', 
# 				'10m_v_component_of_wind',
# 				'total_precipitation'
# 			],
# 			'year': year,
# 			'month': month,
# 			'day': month_days[month],
# 			'time': [
# 				'00:00',
# 				'03:00',
# 				'06:00',
# 				'09:00',
# 				'12:00',
# 				'15:00',
# 				'18:00',
# 				'21:00',
# 			],
# 			'data_format': 'grib',
# 			'download_format': 'unarchived',
# 			'area': [
# 				top_lat, 
# 				left_long, 
# 				bottom_lat, 
# 				right_long,
# 			],
# 			'file_name': grib_filename
# 		})

In [5]:
# ###

# client = None            # source: https://stackoverflow.com/a/64995612
# def set_global_client(): # source: https://stackoverflow.com/a/64995612
#   global client
#   if not client:
#     client = cdsapi.Client(progress = False)
    
# ###

# def cdsapi_worker(dataset):
#   print("retrieving: ", dataset['file_name'])
#   result = client.retrieve('reanalysis-era5-land', dataset)
#   result.download(dataset['file_name'])


# # per-user requests that access online CDS data is 3 at maximum

# pool = multiprocessing.Pool(processes=3, initializer=set_global_client)
# pool.map(cdsapi_worker, datasets)

In [6]:
yearly_dataset = []

# IMPORTANT: Change 'ds_path' to wherever you originally store the data
ds_path = 'C:/Users/sguti/WW-Data/'

for year in years_list:
    # Name of the NetCDF file to convert to after concatenating all monthly datasets
    nc_filename = ds_path + 'Houston_' + year + '.nc'
    
    for month in months_list:
        # Take care of leap years
        if month == '02' and year in leap_years:
            month_days['02'] = feb_days_leap
        else:
            month_days['02'] = feb_days_non_leap
            
        grib_filename = ds_path + 'Houston_' + year + '_' + month + '.grib'
        
        dataset = 'reanalysis-era5-land'
        request = {
            'variable': [
                '2m_temperature', 
                '2m_dewpoint_temperature', 
                'surface_pressure', 
                '10m_u_component_of_wind', 
                '10m_v_component_of_wind',
                'total_precipitation'
            ],
            'year': year,
            'month': month,
            'day': month_days[month],
            'time': [
                '00:00',
                '03:00',
                '06:00',
                '09:00',
                '12:00',
                '15:00',
                '18:00',
                '21:00',
            ],
            'data_format': 'grib',
            'download_format': 'unarchived',
            'area': [
                top_lat, 
                left_long, 
                bottom_lat, 
                right_long,
            ],
        }

        c.retrieve(dataset, request, grib_filename)
        
        # Open the GRIB file. Using 'with' will make sure to close the file before execution leaves the 'with' block.
        with xr.open_dataset(grib_filename, engine = 'cfgrib') as ds:
            # The GRIB file data contains mostly empty labels [number, surface, valid_time]. We'll drop them before we concatenate with the yearly dataset to save space.
            monthly_dataset = ds.drop(['number', 'surface', 'valid_time'], dim=None)
            if len(yearly_dataset) == 0:
                # If yearly_dataset is empty, there's nothing to concatenate with
                yearly_dataset = monthly_dataset
            else: 
                # We will concatenate the two datasets along the time dimension.
                yearly_dataset = xr.concat([yearly_dataset, monthly_dataset], dim="time")

    # Finally, save the yearly_dataset to a NetCDF file. While GRIB is the native format, GRIB data is generally “messier” than data in a self-describing format, such as NetCDF. 
    print("Storing to... ", nc_filename)
    yearly_dataset.to_netcdf(nc_filename)
    yearly_dataset = []

2025-05-25 12:09:01,353 INFO Request ID is 5ed1c126-dc10-4b57-8be4-6da76cdae7c3
2025-05-25 12:09:01,534 INFO status has been updated to accepted
2025-05-25 12:09:34,960 INFO status has been updated to successful


d6185c8bf0b321ed2d894d02ac88327f.grib:   0%|          | 0.00/235k [00:00<?, ?B/s]

Ignoring index file 'C:/Users/sguti/WW-Data/Houston_2008_01.grib.923a8.idx' older than GRIB file
2025-05-25 12:09:41,786 INFO Request ID is 2089969c-8467-4305-a7c3-28e466db1243
2025-05-25 12:09:41,978 INFO status has been updated to accepted
2025-05-25 12:09:57,589 INFO status has been updated to successful


10a00c814529fdb3c96e7b25ec5f59cd.grib:   0%|          | 0.00/220k [00:00<?, ?B/s]

Ignoring index file 'C:/Users/sguti/WW-Data/Houston_2008_02.grib.923a8.idx' older than GRIB file
2025-05-25 12:10:04,048 INFO Request ID is b3b55989-1615-4aa0-bcda-2d6adddfcc6b
2025-05-25 12:10:04,213 INFO status has been updated to accepted
2025-05-25 12:10:13,073 INFO status has been updated to successful


...

2025-05-26 00:00:34,480 INFO Request ID is e867b110-c0e3-4430-961a-c508c890c5f8
2025-05-26 00:00:34,668 INFO status has been updated to accepted
2025-05-26 00:01:08,417 INFO status has been updated to running
2025-05-26 00:04:55,559 INFO status has been updated to successful


51bddef9e4f16ba085348293ff91984c.grib:   0%|          | 0.00/228k [00:00<?, ?B/s]

2025-05-26 00:05:01,405 INFO Request ID is f769d259-2663-4c0e-ad21-88dce4191abf
2025-05-26 00:05:01,611 INFO status has been updated to accepted
2025-05-26 00:05:35,106 INFO status has been updated to running
2025-05-26 00:09:22,235 INFO status has been updated to successful


e0eed3a9adfcb3e0da29f6c3cfd12690.grib:   0%|          | 0.00/235k [00:00<?, ?B/s]

Storing to...  C:/Users/sguti/WW-Data/Houston_2024.nc
